In [19]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD
import plotly.offline
import plotly.graph_objs as go

In [20]:
#df = pd.read_csv("logs_limpios.csv")
df_caja = pd.read_excel("la_caja_logs.xlsx")

In [3]:
#text_list_complete = df_caja["Pregunta"].to_list()

---
> ### 0: ***Preparación de la data para el ploteo***
---

---
> #### 0.1: ***Limpieza de los ejemplos***
---

In [22]:
df_caja["Pregunta"] = df_caja["Pregunta"].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))

In [23]:
def data_plot_image(data_plot, model_cluster, text):
    pos_x = [x[0] for x in data_plot]
    pos_y = [x[1] for x in data_plot]
    
    dic_plot = {}

    for i in range(len(model_cluster)):
        if model_cluster[i] not in dic_plot:
            dic_plot[model_cluster[i]] = [[pos_x[i]],[pos_y[i]], [model_cluster[i]], [text[i]]]
        # Agrego la posición en X
        dic_plot[model_cluster[i]][0].append(pos_x[i])  
        # Agrego la posición en Y
        dic_plot[model_cluster[i]][1].append(pos_y[i])    
        # Agrego el número de "cluster"
        dic_plot[model_cluster[i]][2].append(model_cluster[i])
        # Agrego la pregunta
        dic_plot[model_cluster[i]][3].append(text[i])
        
    # Cantidad de intenciones detectadas
    amount_clusters = len(dic_plot.keys())

    plot = []
    for intents in dic_plot:
        trace = go.Scatter(
            x = dic_plot[intents][0],
            y = dic_plot[intents][1],
            text = dic_plot[intents][3],
            name = str(intents),
            mode = 'markers',
            marker = dict(
                size = 7
            )
        )
        plot.append(trace)
        
    print(f"---> La cantidad de clusters detectados fueron de {amount_clusters}")    
    return plot

---
> ### 1: ***Stemizar y sacar las stopwords del texto***
---

In [34]:
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
import nltk
from nltk.corpus import stopwords

stemmer = SnowballStemmer('spanish')
cachedStopWords = stopwords.words('spanish')

stopwords_nico = ['buenos','dias', 'hola', 'gracias', 'muchas', 'ok']
cachedStopWords.extend(stopwords_nico)


def clean_text(text_list):
    for i in range(len(text_list)):
        text = ' '.join([word for word in text_list[i].split() if word.lower() not in cachedStopWords]) #[stemmer.stem(word) for word in text_list[i].split() if word not in cachedStopWords])
        text_list[i] = text
    return text_list

In [35]:
text_list = clean_text(text_list_complete)

---
> ### 2: ***Vectorizar el texto utilizando Tf-idf (TfidfVectorizer) o BOW (CountVectorizer)***

### Diferencias entre las dos
---

In [36]:
vectorizer_tfidf = TfidfVectorizer(ngram_range=(1,2))
vectorizer_bow = CountVectorizer(ngram_range=(1,1))

In [37]:
data = vectorizer_tfidf.fit_transform(text_list)

---
> ### ***Gráfico de energía***
---

In [ ]:
from scipy.sparse.linalg import eigs

matrix = np.matmul(data.toarray(), data.toarray().transpose())
vals, vecs = eigs(matrix, 20)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
plt.plot(vals);

---
> ### 3: ***Aplicar SVD/PCA y reducir las dimensiones dependiendo del gráfico de energía***
---

In [38]:
svd = TruncatedSVD(n_components=3)
svd_truncated = svd.fit_transform(data)

---
> ### 3.1: ***Aplicar T-SNE para reducir todo a 2 dimensiones***
---

In [39]:
# method = 'exact' corre un algorito más exacto pero de complejidad O(N^2)

data_2d = TSNE(n_components=2, n_iter=1000, metric='cosine')
data_2d = data_2d.fit_transform(svd_truncated)
data_plot = data_2d

---
> ### 4: ***Ploteo la data para ver qué algoritmo de clustering utilizar***
---

In [40]:
plot = []
pos_x = [x[0] for x in data_plot]
pos_y = [x[1] for x in data_plot]

trace = go.Scatter(
    x = pos_x,
    y = pos_y,
    text = df_caja["Pregunta"].to_list(),
    mode = 'markers',
    marker = dict(
        size = 4
    )
)
plot.append(trace)
plotly.offline.plot(plot)

'file:///Users/nicolas.bugliot/Documents/Trabajo/ML/Clustering/temp-plot.html'

---
> ### Tipo de clustering: ***K-MEANS++***
---

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=15, random_state=0).fit_predict(data_plot)

In [240]:
plotly.offline.plot(data_plot_image(data_plot, kmeans, text_list_complete));

---> La cantidad de clusters detectados fueron de 15


---
> ### Tipo de clustering: ***Spectral Clustering***
---

In [236]:
%%time
from sklearn.cluster import SpectralClustering
spectral_clustering = SpectralClustering(n_clusters=15, assign_labels="discretize", random_state=0).fit(data_plot)

CPU times: user 35.7 s, sys: 6.44 s, total: 42.1 s
Wall time: 10.7 s


In [241]:
plotly.offline.plot(data_plot_image(data_plot, spectral_clustering.labels_, text_list_complete));

---> La cantidad de clusters detectados fueron de 15


---
> ### Tipo de clustering: ***DBSCAN***
---

In [242]:
from sklearn.cluster import DBSCAN

In [247]:
# eps es el hiper-parámetro que define el radio del radar.
dbscan_clustering = DBSCAN(eps=4, min_samples=15).fit(data_plot)

In [248]:
plotly.offline.plot(data_plot_image(data_plot, dbscan_clustering.labels_, text_list_complete));

---> La cantidad de clusters detectados fueron de 7


---
> ### Tipo de clustering: ***HDBSCAN***
---

In [249]:
import hdbscan

In [250]:
%%time
hdbscan_cluster = hdbscan.HDBSCAN()
hdbscan_cluster = hdbscan_cluster.fit(data_plot)

CPU times: user 39.8 ms, sys: 6.94 ms, total: 46.7 ms
Wall time: 45.6 ms


In [251]:
plotly.offline.plot(data_plot_image(data_plot, hdbscan_cluster.labels_, text_list_complete));

---> La cantidad de clusters detectados fueron de 17


---
> ### Clustering: ***Datos en N dimensiones***
---

In [342]:
data_complete = vectorizer_tfidf.fit_transform(text_list)

In [343]:
svd_complete = TruncatedSVD(n_components=30, n_iter=100)
svd_truncated = svd_complete.fit_transform(data_complete)

In [344]:
from sklearn.cluster import DBSCAN
# eps es el hiper-parámetro que define el radio del radar.
dbscan_complete = DBSCAN(eps=0.1, min_samples=10).fit(svd_truncated)

In [345]:
import hdbscan
hdbscan_cluster = hdbscan.HDBSCAN()
hdbscan_cluster_complete = hdbscan_cluster.fit(svd_truncated)

In [346]:
a = dbscan_complete.labels_

In [347]:
l = []
for i in a:
    if i not in l:
        l.append(i)

In [348]:
l

[-1, 0, 1, 5, 2, 3, 4, 6]

In [349]:
for i in dbscan_complete.labels_:
    #print(i)

SyntaxError: unexpected EOF while parsing (<ipython-input-349-3ea0f086fcd5>, line 2)

In [350]:
dataframe_preguntas = pd.DataFrame(text_list_complete)
dataframe_preguntas["cluster"] = dbscan_complete.labels_

In [351]:
dataframe_preguntas["cantidad"] = 1

In [352]:
dataframe_preguntas.groupby(["cluster", 0]).agg({"cantidad":"sum"})

cantidad
cluster 0                                                           
-1      - consulta promociones articulos ocn compra 100...         1
        -quiero retirarlo pick up point alto palermo               1
        . acabo hacer compra on line retirar boutique a...         1
        . acabo registrar ahora puede ingresar usuario             1
        . algun inconveniente ingresarponfgo mail contr...         1
        . ayer local unicenter mandaron mail link debo ...         1
        . ayer realice compra tomaron datos.me dijeron ...         1
        . comprando 100 cápsulas permite hacer envío do...         1
        . compre maquina mayo. quiero hacer pedido teng...         1
        . compré primera caffetera inissio fravega aero...         1
        . creo haber olvidado clave ingreso. podrán env...         1
        . cuál num código promocional?                             1
        . código promocional cliente frecuente?                    1
        . código promocional pide?                                 1
        . deja seleccionar retirar pedido boutique palermo         1
        . essenza type c90 cuyo boton encendido hunde a...         1
        . hace bastante puedo entrar cuenta internet ha...         1
        . hace semana compre capsulas entregaron aunman...         1
        . hacer pedido. factura quieren cobrar percepci...         1
        . haciendo pedido retirar alto palermo. necesit...         1
        . haciendo pedido retirar boutique nepresso enc...         1
        . hago pedido mas 100 capsulas general pedido a...         1
        . hago pregunta. reciencompre 150 capsulas pagi...         1
        . hago.para obtener cupon descuento. recién mie...         1
        . hice pedido hago pasar pago tarjeta q permite...         1
        . intentando definir pick-up point garbarino no...         1
        . intentando entrar ususario indica contraseña ...         1
        . llego mail anuncia descuento $ 200 pagar aparece         1
        . llegó mail informan comprando 150 cápsulas gr...         1
        . mail promoción dice descuento 200 $ compras r...         1
...                                                              ...
 5      atascado completer pedido                                  1
        bien viendo armar pedido consulta                          1
        buenasquisiera realizar pedido pasarlo buscar y...         1
        deseo pedido entregado cot 103 colon rios                  1
        despues pedido ir                                          1
        estas?? pedido recibiria dia lunes ?                       1
        finalizo pedido                                            1
        haciendo pedido café. marco pedido retiren bols...         1
        haga cerrar pedido ?                                       1
        jonatan molesto pues procesa pedido                        1
        llego pedidoestimados . llegado pedido solicite...         1
        modificar pedido                                           1
        pedido cobran ingresos brutos?                             1
        pedido llevan casa retirarlo                               1
        pedido regalo ?me dijeron porta cápsulascompran...         1
        posibilidad cambiar pedido?                                1
        quise modificar pedido desapareció                         1
        r favor enviar pedido dirección particular”barr...         1
        realizando pedido tildo paso 3                             1
        retire tercero pedido ?                                    1
 6      algun codigo promocional?                                  1
        buscando codigo promocional                                1
        codigo promocional es?                                     1
        codigo promocional? q piden?                               1
        cuál codigo promocional                                    1
        falta codigo promocional...                       